<a href="https://colab.research.google.com/github/fedyasa/CSC/blob/main/FedorSergeev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
         
fantasy_zip = zipfile.ZipFile('/content/train.zip')
fantasy_zip.extractall('/content/train')
 
fantasy_zip.close()

In [2]:
import pandas as pd
import numpy as np

In [3]:
my_dict = {}

df = pd.read_csv('/content/train_tags.csv', header=None) 
my_dict = dict(zip(df[0], df[1]))

In [4]:
op = []
sg = []

for id in my_dict:
  if my_dict[id] == 'open':
    op.append(id)
  if my_dict[id] == 'sunglasses':
    sg.append(id)

In [5]:
import os
import shutil

os.mkdir('/content/val')
os.mkdir('/content/val/opn')
os.mkdir('/content/val/sun')
os.mkdir('data')
os.mkdir('/content/data/opn')
os.mkdir('/content/data/sun')

In [6]:
path = r"/content/train/train"

for file in os.listdir(path):
  if os.path.splitext(file)[0] in op:
    shutil.move(path + '/' + file, '/content/data/opn/' + file)
  if os.path.splitext(file)[0] in sg:
    shutil.move(path + '/' + file, '/content/data/sun/' + file)

In [7]:
import random
from tqdm import tqdm

for file in tqdm(os.listdir('/content/data/opn')):
  if random.uniform(0, 7) > 6:
    shutil.move('/content/data/opn/' + file, '/content/val/opn/' + file)

for file in tqdm(os.listdir('/content/data/sun')):
  if random.uniform(0, 7) > 6:
    shutil.move('/content/data/sun/' + file, '/content/val/sun/' + file)

100%|██████████| 189/189 [00:00<00:00, 140653.56it/s]


In [8]:
import torchvision
from torchvision import transforms, models
import torch

f = 25

train_transforms_0 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

train_transforms_1 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

train_transforms_2 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

train_transforms_3 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

train_transforms_4 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

train_transforms_5 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

train_transforms_6 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomRotation(50),
    transforms.ToTensor()
])

train_transforms_7 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomRotation(50),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

train_transforms_8 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

train_transforms_9 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

train_transforms_10 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(50),
    transforms.ToTensor()
])


train_transforms_11 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(50),
    transforms.CenterCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

train_transforms_12 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(50),
    transforms.RandomCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

train_transforms_13 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomCrop(f),
    transforms.Resize((30, 32)),
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

data = '/content/data'
val_dir = '/content/val'

train_dataset_0 = torchvision.datasets.ImageFolder(data, train_transforms_0)
train_dataset_1 = torchvision.datasets.ImageFolder(data, train_transforms_1)
train_dataset_2 = torchvision.datasets.ImageFolder(data, train_transforms_2)
train_dataset_3 = torchvision.datasets.ImageFolder(data, train_transforms_3)
train_dataset_4 = torchvision.datasets.ImageFolder(data, train_transforms_4)
train_dataset_5 = torchvision.datasets.ImageFolder(data, train_transforms_5)
train_dataset_6 = torchvision.datasets.ImageFolder(data, train_transforms_6)
train_dataset_7 = torchvision.datasets.ImageFolder(data, train_transforms_0)
train_dataset_8 = torchvision.datasets.ImageFolder(data, train_transforms_1)
train_dataset_9 = torchvision.datasets.ImageFolder(data, train_transforms_2)
train_dataset_10 = torchvision.datasets.ImageFolder(data, train_transforms_3)
train_dataset_11 = torchvision.datasets.ImageFolder(data, train_transforms_4)
train_dataset_12 = torchvision.datasets.ImageFolder(data, train_transforms_5)
train_dataset_13 = torchvision.datasets.ImageFolder(data, train_transforms_6)

val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

train_dataset = torch.utils.data.ConcatDataset([train_dataset_0, train_dataset_1,
                                                train_dataset_2, train_dataset_3,
                                                train_dataset_4, train_dataset_5,
                                                train_dataset_6, train_dataset_7,
                                                train_dataset_8, train_dataset_9,
                                                train_dataset_10, train_dataset_11,
                                                train_dataset_12, train_dataset_13
                                               ])

batch_size = 10

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle = True)

val_dataloader = torch.utils.data.DataLoader(val_dataset)

In [9]:
len(train_dataloader), len(val_dataloader)

(451, 56)

In [27]:
from torch.nn.modules.loss import CrossEntropyLoss
import torch.nn as nn
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.L1 = nn.Linear(30*32, 256)
    self.R1 = nn.ReLU()
    self.L2 = nn.Linear(256, 64)
    self.R2 = nn.ReLU()
    self.L3 = nn.Linear(64, 2)
  def forward(self, input):
    x = self.L1(input)
    x = self.R1(x)
    x = self.L2(x)
    x = self.R2(x)
    x = self.L3(x)
    return x

model = Model()

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0003)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [32]:
def train(model, loss, optimizer, num_epochs = 25):

  for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                model.train()
            else:
                dataloader = val_dataloader
                model.eval()

            running_loss = 0.
            running_acc = 0.

            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                inputs = inputs.reshape(inputs.shape[0], -1)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

  return model

In [29]:
train(model, loss, optimizer)

Epoch 0/24:


100%|██████████| 451/451 [00:03<00:00, 133.75it/s]

train Loss: 0.6217 Acc: 0.6529



100%|██████████| 56/56 [00:00<00:00, 1263.40it/s]

val Loss: 0.5256 Acc: 0.8036
Epoch 1/24:


...


100%|██████████| 451/451 [00:04<00:00, 101.19it/s]

train Loss: 0.0447 Acc: 0.9863



100%|██████████| 56/56 [00:00<00:00, 1037.14it/s]

val Loss: 0.1609 Acc: 0.9464


Model(
  (L1): Linear(in_features=960, out_features=256, bias=True)
  (R1): ReLU()
  (L2): Linear(in_features=256, out_features=64, bias=True)
  (R2): ReLU()
  (L3): Linear(in_features=64, out_features=2, bias=True)
)

In [33]:
fantasy_zip = zipfile.ZipFile('/content/test.zip')
fantasy_zip.extractall('/content')
 
fantasy_zip.close()

In [34]:
from PIL import Image
a = iter(os.listdir("/content/test"))
file = next(a)
image_input = val_transforms(Image.open("/content/test/" + file)).to(device)
image_input.shape
model.eval()
model(image_input.reshape(-1))

tensor([ 1.0595, -1.0780], grad_fn=<AddBackward0>)

In [35]:
test_predictions = []
test_img_paths = []

for file in tqdm(os.listdir("/content/test")):
    inputs = val_transforms(Image.open("/content/test/" + file)).to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs.reshape(-1))
    test_predictions.append(
        torch.nn.functional.softmax(preds).data.cpu().numpy())
    test_img_paths.append(file)

  0%|          | 0/22140 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
100%|██████████| 22140/22140 [00:13<00:00, 1702.60it/s]


In [36]:
print(len(test_img_paths))
print(len(test_predictions))

22140
22140


In [37]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'sunglasses_probability': test_predictions})

In [38]:
submission_df

,id,sunglasses_probability
0,19529.jpg,"[0.894496, 0.10550396]"
1,7696.jpg,"[0.016075987, 0.9839241]"
2,19554.jpg,"[0.8571234, 0.14287664]"
3,16112.jpg,"[0.005909881, 0.99409014]"
4,13624.jpg,"[0.0654729, 0.9345271]"
...,...,...
22135,21615.jpg,"[0.99915326, 0.0008467238]"
22136,10164.jpg,"[1.1534627e-06, 0.9999988]"
22137,19661.jpg,"[0.00080423534, 0.99919575]"
22138,9709.jpg,"[0.11657473, 0.8834253]"


In [39]:
os.mkdir('/content/result')

In [40]:
for i in range(0, 22140):
  submission_df['sunglasses_probability'][i] = submission_df['sunglasses_probability'][i][1]
#.map(lambda pred: 'open' if pred[0] > 0.5 else 'sunglasses')
submission_df['id'] = submission_df['id'].str.replace('/content/result/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,sunglasses_probability
id,
19529,0.105504
7696,0.983924
19554,0.142877
16112,0.99409
13624,0.934527
5882,0.000068


In [41]:
submission_df.index = submission_df.index.astype(int)
submission_df = submission_df.sort_index()
submission_df.head(n=6)

,sunglasses_probability
id,
0,0.05439
1,0.960955
2,0.982766
3,0.000001
4,0.0
5,0.000018


In [42]:
submission_df.to_csv('Fedor_Sergeev.csv')